In [ ]:
import sys
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

import re, string, unicodedata
import nltk
#import contractions
import warnings
warnings.filterwarnings('ignore')
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud

In [ ]:
reddit_data = pd.concat(map(pd.read_csv, ['subreddits_data.csv']))
reddit_data.info()

In [ ]:
reddit_data.head()

In [ ]:
#In this dataset there are several columns of type string but they must be numerical.
numeric_columns= ['score','comms_num','author_comments_karma', 'author_links_karma']
for column in numeric_columns:
    reddit_data[column]=pd.to_numeric(reddit_data[column], errors='coerce')

In [ ]:
# Column timestamp is of type string, let us convert it to type timestamp
reddit_data['timestamp']=pd.to_datetime(reddit_data['timestamp']) 
# Number of reddits made in each hour of the day.
reddit_data['reddit_created_hour']=reddit_data['timestamp'].apply(lambda time: time.hour)

In [ ]:
reddit_data.info()

In [ ]:
#Let's see the distribution of reddits for each hour.
sns.catplot(x="reddit_created_hour", kind="count", data=reddit_data)
plt.show()


Here we can see that most reddits are published between 3 p.m. and 7 p.m., which is not a surprise.
Now, let us see the distribution of how much time has elapsed since each reddit was created(measured in days).

In [ ]:
import datetime
#Time interval(measured in hours) since the reddits was created until now.
reddit_data['elapsed_hours']=reddit_data.timestamp.\
apply(lambda timestamp: (datetime.datetime.now()-timestamp)/np.timedelta64(1,'h'))

In [ ]:
first_percentile =np.percentile(reddit_data['elapsed_hours'],25)
median =np.percentile(reddit_data['elapsed_hours'],50)
third_percentile =np.percentile(reddit_data['elapsed_hours'],75)
sns.distplot(reddit_data['elapsed_hours'], bins=20, kde=False, rug=False)

plt.axvline(first_percentile, color="green",linestyle="--",label="25%_percentile")
plt.axvline(median, color="red",linestyle="--",label="median")
plt.axvline(third_percentile, color="green",linestyle="--",label="75%_percentile")
plt.title('Counting of elapsed hours since reddits creation')
plt.xlabel('Elapsed_hours')
plt.ylabel('Count');
plt.legend()

plt.show()

In [ ]:
print("Min and Max hours elapsed since reddit creation, min: {:.0f}, max: {:.0f}."\
      .format(reddit_data['elapsed_hours'].min(),reddit_data['elapsed_hours'].max()))

Quantities such as the score and the number of comments grow over time. Then, the proper way to use these features is to normalize them with respect to elapsed_hours.

In [ ]:
# Para hacer el grafico de la frecuencia de los datos numericos primero normalizemos estas features
from sklearn.preprocessing import MinMaxScaler
# MinMaxScaler instance
scaler = MinMaxScaler()
#This estimator scales and translates each feature individually 
# From sklearn documentation, the transformation is calculated as follow:
#X_scaled = scale * X + min - X.min(axis=0) * scale
#where scale = (max - min) / (X.max(axis=0) - X.min(axis=0))
# Numeric columns
reddits_standardize =reddit_data[['score','comms_num','author_comments_karma', 'author_links_karma']]
# Taking into account elapsed_hours
reddits_standardize=reddits_standardize.div(reddit_data['elapsed_hours'],axis=0)

# Remove null values
reddits_standardize = reddits_standardize.dropna()
# Fit and transform
X_standardize =scaler.fit_transform(reddits_standardize)
# Dataframe with numeric features
data_standardize = pd.DataFrame(X_standardize,columns=['score','comms_num','author_comments_karma', \
                                                      'author_links_karma'])

In [ ]:
data_standardize.head()

In data_standardize, the values of each feature is normalized to the interval $[0,1]$. But, I am also
interested in exploring the behavior of the logarithm of these columns.
Since logarithm is undefined at zero, let's shift the min of each feature towards $1.0$.

In [ ]:
shifted_data_standardize =data_standardize+1.0
log_data_standardize=np.log(shifted_data_standardize)
log_data_standardize.columns=list(map(lambda x: 'log_'+x, data_standardize.columns))
shifted_data_standardize.head()


In [ ]:
# Function that plots histograms of numerical columns.

def plot_histogram(dataframe,column,y_label,title,bins=8):
    '''
    Take a column and plot two histograms, where:
    left: Distribution of column values
    right: Distribution of log of column values
    
    '''
    # Params:
    
    # dataframe is the data frame to plot
    # column is the numeric feature 
    # y_label is the y label 
    # title is the title of the plot
    # bins is the number of bins
     
    
    #Return : 
    
    #Two, left: column values, right: logarithm of column values
    
    fig, (ax1, ax2) = plt.subplots(1,2,figsize=(8, 6),sharey=True)
    fig.suptitle(title)
    # Feature values 
    ax1.hist(dataframe[column],bins=bins,log=True)
    #Log of feature values 
    ax2.hist(np.log(dataframe[column]),bins=bins,log=True)
    # Labels
    ax1.set_xlabel(column, labelpad=15, fontsize=12, color="black");
    ax1.set_ylabel(y_label, labelpad=15, fontsize=12, color="black");
    ax2.set_xlabel("Log of "+ column, labelpad=15, fontsize=12, color="black");
    
    #ax.title.set_text(title)
    #ax.set_axisbelow(True)
    plt.grid(True, alpha=0.5)
    

In [ ]:
# Histograms of the columns
for column in  shifted_data_standardize.columns:
    plot_histogram(shifted_data_standardize,column,"Frequency (Log scale)", "Histogram of " + column)

The graphics above show very skewed distributions for all features. In fact, this behavior occurs in the plots of the logarithms too. Now, let us analyze the correlation among the  features(in log scale).

In [ ]:
log_data_standardize.corr()

In [ ]:
# Generating correlation heatmap 
sns.heatmap(log_data_standardize.corr(), annot = True)  
plt.show() 

Here we have two goups of features. The first group is associated with the reddit author, these features are author_comments_karma	and author_links_karma.The above map suggest a kind of correlation between them. 
The second group is associated with the reddit itself, these are score	and comms_num, in this case there is a strong correlation between them.

In [ ]:
# Regression line for features log_score and log_comms_num
g = sns.jointplot(x=log_data_standardize["log_score"],\
                  y=log_data_standardize["log_comms_num"],\
                  data=log_data_standardize,kind='reg')

As I said before, log_score and  log_comms_num are strongly correlated.

In [ ]:
# Regression line for features log_links_karma" and "log_comments_karma
g = sns.jointplot(x=log_data_standardize["log_author_comments_karma"],\
                  y=log_data_standardize["log_author_links_karma"],\
                  data=log_data_standardize,kind='reg')


Here, log_author_comments_karma and log_author_links_karma show a weak correlation between them.

In [ ]:
# Regression line for features log_score  and log_author_links_karma
g = sns.jointplot(x=log_data_standardize["log_score"],\
                  y=log_data_standardize["log_author_links_karma"],\
                  data=log_data_standardize,kind='reg')

Here, the correlation is week too.

Let us see if a 3d plot allows us to detect any pattern between these features. 

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(12,10))
ax = fig.add_subplot(111, projection='3d')

x= log_data_standardize["log_author_comments_karma"]
y=log_data_standardize["log_author_links_karma"]
z= log_data_standardize["log_score"]
ax.scatter(x, y, z, s=50)
fig.suptitle('comments_karma vs links_karma vs score in log scale')

ax.set_xlabel('log comments_karma')
ax.set_ylabel('log links_karma')
ax.set_zlabel('log score')

plt.show()
               

Almost all points are grouped at the origin and there are several outliers.
Let us see if reddits are organized by clusters.
First, let us remove the superfluous features,so,  let us use a dimensional reduction technique such as
principal component analysis.

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

X_standardize=log_data_standardize.values

# Create scaler: scaler
scaler = StandardScaler()

# Create PCA instance: pca
pca = PCA()

# Create pipeline: pipeline
pipeline = make_pipeline(scaler,pca)

#Fit and transform the pipeline to samples
pipeline.fit_transform(X_standardize)

# Plot percent of variance of each pca_feature
plt.figure()
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of Components')
plt.ylabel('Variance (%)') #for each component
plt.title('Reddit features Explained Variance')
plt.show()


From the previous graph, almost $100$% of the variation is explained by $2$ characteristics.

In [ ]:
# Create PCA instance: pca
pca = PCA(n_components=2)

# Create pipeline: pipeline
pipeline = make_pipeline(scaler,pca)
pipeline.fit(X_standardize)
# The pca features
Xpca = pipeline.transform(X_standardize)

# Plot of datapoints using pca_features
sns.set()
plt.figure(figsize=(8,8))
plt.scatter(Xpca[:,0],Xpca[:,1])
plt.xlabel("PCA_feature_1")
plt.ylabel("PCA_feature_2")
plt.show()

Again, we can see several outliers.
Let us explore some clustering techniques to analyze if our data has clustering structure.
Let us begin with the most popular one: K-Means. Essentially, K-Means sets k centroids in the data and clusters points by assigning them to the nearest centroid.  A way to select the optimal number of clusters is by using the elbow method. At high level, the idea is to find the least number of cluster where you can see  an elbow in the plot of number of clusters vs inertia(check this link https://www.scikit-yb.org/en/latest/api/cluster/elbow.html).

In [ ]:
from sklearn.cluster import KMeans
range_n_clusters = range(2,15)
inertias=[]

for k in range_n_clusters: 
    #Building and fitting the model for the chosen number of clusters 
    clusterer_kmeans = KMeans(n_clusters=k)
    cluster_labels = clusterer_kmeans.fit_predict(Xpca)
    # Adding inertias to a list 
    inertias.append(clusterer_kmeans.inertia_) 
    
# Number of clusters vs Inertias   
plt.plot(range_n_clusters, inertias, 'bo-') 
plt.xlabel('Number of clusters') 
plt.ylabel('Inertia') 
plt.title('The Elbow Method using Inertia') 
plt.show() 

The elbow method suggests us that there are 2-6 clusters in the dataset.
Let us explore the silhouette method for different clustering algorithms.
The silhouette value determines how similar a given point  is to its own cluster compared to other clusters. The Silhouette values  are in the interval $[-1,1]$, where $1$ indicates that the point is well matched to its own cluster and poorly matched to neighboring clusters. If most pints  have a high value, then the clustering configuration is appropriate. 

In [ ]:
from sklearn.cluster import AgglomerativeClustering as AggClus
from sklearn.cluster import SpectralClustering as SpectClus
from sklearn.metrics import silhouette_samples, silhouette_score
#import sys
import warnings
warnings.filterwarnings('ignore')
import matplotlib.cm as cm
import re 



def silhouette_plots(range_n_clusters,X,method=KMeans):
    # Params:
    # range_n_clusters is a list containing the different cluster numbers that we are going to analyze.
    # X is the  data
    # method is the clustering method
    
    # Return:
    # Two plots, for each number of clusters, left plot: silhouette_score 
    #and right plot: is the scatter plot of our features 
    
    for n_clusters in range_n_clusters:
        # Create a subplot with 1 row and 2 columns
        fig, (ax1, ax2) = plt.subplots(1, 2)
        fig.set_size_inches(14, 6)

        # The 1st subplot is the silhouette plot
        # The silhouette coefficient can range from -1, 1 but in this example all
        # lie within [-0.4, 1]
        ax1.set_xlim([-0.6, 1])
        # The (n_clusters+1)*10 is for inserting blank space between silhouette
        # plots of individual clusters, to demarcate them clearly.
        ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

        # Initialize the clusterer with n_clusters value 
        clusterer = method(n_clusters=n_clusters)
        cluster_labels = clusterer.fit_predict(X)

        # The silhouette_score gives the average value for all the samples.
        # This gives a perspective into the density and separation of the formed
        # clusters
        silhouette_avg = silhouette_score(X, cluster_labels)
        print("For n_clusters =", n_clusters,
              "The average silhouette_score is :", silhouette_avg)

        # Compute the silhouette scores for each sample
        sample_silhouette_values = silhouette_samples(X, cluster_labels)
        

        y_lower = 10
        for i in range(n_clusters):
            # Aggregate the silhouette scores for samples belonging to
            # cluster i, and sort them
            ith_cluster_silhouette_values = \
                sample_silhouette_values[cluster_labels == i]

            ith_cluster_silhouette_values.sort()

            size_cluster_i = ith_cluster_silhouette_values.shape[0]
            y_upper = y_lower + size_cluster_i

            color = cm.nipy_spectral(float(i) / n_clusters)
            ax1.fill_betweenx(np.arange(y_lower, y_upper),
                              0, ith_cluster_silhouette_values,
                              facecolor=color, edgecolor=color, alpha=0.7)

            # Label the silhouette plots with their cluster numbers at the middle
            ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

            # Compute the new y_lower for next plot
            y_lower = y_upper + 10  # 10 for the 0 samples

        ax1.set_title("The silhouette plot for the various clusters.")
        ax1.set_xlabel("The silhouette coefficient values")
        ax1.set_ylabel("Cluster label")

        # The vertical line for average silhouette score of all the values
        ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

        ax1.set_yticks([])  # Clear the yaxis labels / ticks
        ax1.set_xticks([-0.6,-0.4,-0.2, 0, 0.2, 0.4, 0.6, 0.8, 1])

        # 2nd Plot showing the actual clusters formed
        colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
        ax2.scatter(X[:, 0], X[:, 1], marker='.', s=30, lw=0, alpha=0.7,
                    c=colors, edgecolor='k')

        # Labeling the clusters
        if method == KMeans:
            centers = clusterer.cluster_centers_
            #Draw white circles at cluster centers
            ax2.scatter(centers[:, 0], centers[:, 1], marker='o',\
                        c="white", alpha=1, s=200, edgecolor='k')

            for i, c in enumerate(centers):
                ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1,\
                            s=50, edgecolor='k')

        ax2.set_title("The visualization of the clustered data.")
        ax2.set_xlabel("Feature space for the 1st feature")
        ax2.set_ylabel("Feature space for the 2nd feature")
        # Get the name of the clustering method
        method_name=str(method().__class__)
        # Extract only the name 
        str_method= re.findall(r".[\w]+'", method_name)[0][1:-1]
        
        plt.suptitle(("Silhouette analysis for " + str_method + " on sample data with n_clusters = %d" % n_clusters),
                     fontsize=14, fontweight='bold')
        
        

    plt.show()

In [ ]:
range_n_clusters = [2,3,4,6]

In [ ]:
silhouette_plots(range_n_clusters,Xpca,KMeans);

In [ ]:
silhouette_plots(range_n_clusters,Xpca,SpectClus) 

In [ ]:
silhouette_plots(range_n_clusters,Xpca,AggClus) 

By using various clustering methods, the silhouette score suggests that the optimal number of clusters is 2.
Let us study  density-based clustering, this kind of algorithms  are good identifying data points that deviate from the normal distribution, in other words, outliers. A popular density based method is DBSCAN. This method  does not require the number of clusters as a parameter. Here, the main parameters are the radius of the neighborhoods and the minimum number points you want in a neighborhood to define a cluster.

In [ ]:
from sklearn.cluster import DBSCAN
# Object dbscan
dbsc = DBSCAN(eps = .8, min_samples = 10)
# fit and transform
labels_dbsc = dbsc.fit_predict(Xpca)


In [ ]:
plt.scatter(Xpca[:,0],Xpca[:,1],c=labels_dbsc);

Let us see how these two clusters look in our original data.

In [ ]:
fig = plt.figure(figsize=(12,10))
ax = fig.add_subplot(111, projection='3d')

x= log_data_standardize["log_author_comments_karma"]
y=log_data_standardize["log_author_links_karma"]
z= log_data_standardize["log_score"]
ax.scatter(x, y, z, s=50,c=labels_dbsc)
fig.suptitle('Two clusters: comments_karma vs links_karma vs score in log scale')

ax.set_xlabel('log comments_karma')
ax.set_ylabel('log links_karma')
ax.set_zlabel('log score')

plt.show()




Let us see what are the most used words in  reddits that are outliers.

In [ ]:
reddits_not_null=reddit_data[~reddit_data.author_comments_karma.isnull()]
# New column with cluster labels
reddits_not_null['is_outliers'] =labels_dbsc

In [ ]:
reddits_not_null['is_outliers'].value_counts()

There are 32 outliers in our dataset.

In [ ]:
# Outliers reddits 
reddits_not_null[reddits_not_null['is_outliers']==-1]

Before exploring the most common words used in the reddits for each cluster, let us  normalize the text of the titles.

In [ ]:
def normalization(words):
    #Params: words is the words in title
    
    #Return: normalized list of words
    
    list_words = []
    #lemmatizer
    lemmatizer = WordNetLemmatizer()
    for word in words:
        #Remove non-ASCII characters from the word
        word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        # Convert all characters to lowercase
        word = word.lower()
        #Removing punctuation
        word = re.sub(r'[^A-Za-z\s]', '', word)
        #Removing stop words
        if word != '' and  word not in stopwords.words('english'):
            #word = stemmer.stem(word)
            # Eliminating short words and maintaining negations
            if (word !='no' or word !='nt')  and len(word)>2:
                word = lemmatizer.lemmatize(word, pos='v')
            
            list_words.append(word)
    return list_words


In [ ]:
# Words tokenizer
tokenized_title=reddits_not_null.title.apply(lambda entry: nltk.word_tokenize(entry))

In [ ]:
# Normalization of titles
clean_titles=tokenized_title.apply(lambda entry: normalization(entry))

In [ ]:
#Transforming each entry of clean_titles from type list to string.
reddits_not_null['clean_title']=clean_titles.apply(lambda x: " ".join(x))
reddits_not_null.head()

In [ ]:
# Most used words for outliers 
from wordcloud import WordCloud, STOPWORDS
word_cloud = WordCloud(stopwords=STOPWORDS)
plt.figure(figsize=(14, 14))
word_cloud.generate(str(reddits_not_null[reddits_not_null['is_outliers']==-1].clean_title))
plt.imshow(word_cloud)
plt.title('Most common words for outliers')
plt.show()

In [ ]:
# Most used words for non outliers 
word_cloud = WordCloud(stopwords=STOPWORDS)
plt.figure(figsize=(14, 14))
word_cloud.generate(str(reddits_not_null[reddits_not_null['is_outliers']==0].clean_title))
plt.imshow(word_cloud)
plt.title('Most common words for outliers')
plt.show()